In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.spatial import distance
from tqdm import notebook
import pickle
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('New_Flats.csv')
df

,Unnamed: 0,Количество комнат,Цена квартиры,Площадь,Район,Ссылка
0,0,1,2846000,27.10,Академический,https://www.an-nks.ru/search?view=filter-searc...
1,1,1,2946000,27.10,Академический,https://www.an-nks.ru/search?view=filter-searc...
2,2,1,3200000,30.80,Академический,https://www.an-nks.ru/search?view=filter-searc...
3,3,1,3350000,35.45,Академический,https://www.an-nks.ru/search?view=filter-searc...
4,4,1,3455000,34.95,Академический,https://www.an-nks.ru/search?view=filter-searc...
...,...,...,...,...,...,...
699,699,3,10000000,108.76,Ленинский,https://www.an-nks.ru/search?view=filter-searc...
700,700,3,10300000,108.10,Ленинский,https://www.an-nks.ru/search?view=filter-searc...
701,701,3,10500000,108.28,Ленинский,https://www.an-nks.ru/search?view=filter-searc...
702,702,4,10600000,114.30,Ленинский,https://www.an-nks.ru/search?view=filter-searc...


In [3]:
df = df.drop('Unnamed: 0', axis=1)

In [4]:
def no_space(word):
    for i in range(len(word)):
        if word[i] == ' ':
            word = word[0:i:1] + '_' + word[i+1:len(word):1]
    return word

In [5]:
new_names = []
for name in df.columns:
    val = no_space(name.lower())
    new_names.append(val)
new_names

['количество_комнат', 'цена_квартиры', 'площадь', 'район', 'ссылка']

In [6]:
df.set_axis(new_names, axis=1, inplace=True)
df

,количество_комнат,цена_квартиры,площадь,район,ссылка
0,1,2846000,27.10,Академический,https://www.an-nks.ru/search?view=filter-searc...
1,1,2946000,27.10,Академический,https://www.an-nks.ru/search?view=filter-searc...
2,1,3200000,30.80,Академический,https://www.an-nks.ru/search?view=filter-searc...
3,1,3350000,35.45,Академический,https://www.an-nks.ru/search?view=filter-searc...
4,1,3455000,34.95,Академический,https://www.an-nks.ru/search?view=filter-searc...
...,...,...,...,...,...
699,3,10000000,108.76,Ленинский,https://www.an-nks.ru/search?view=filter-searc...
700,3,10300000,108.10,Ленинский,https://www.an-nks.ru/search?view=filter-searc...
701,3,10500000,108.28,Ленинский,https://www.an-nks.ru/search?view=filter-searc...
702,4,10600000,114.30,Ленинский,https://www.an-nks.ru/search?view=filter-searc...


In [7]:
df['район'].value_counts()

Чкаловский         349
Октябрьский        100
Железнодорожный     98
Верх-Исетский       79
Ленинский           51
Академический       27
Name: район, dtype: int64

In [8]:
def category(value):
    if value < 6:
        res = 'дешевая'
    elif value < 8:
        res = 'средняя'
    else:
        res = 'премиальная'
    return res

In [9]:
df['цена_квартиры'] = round(df['цена_квартиры'] / 10**6, 2)
df

,количество_комнат,цена_квартиры,площадь,район,ссылка
0,1,2.85,27.10,Академический,https://www.an-nks.ru/search?view=filter-searc...
1,1,2.95,27.10,Академический,https://www.an-nks.ru/search?view=filter-searc...
2,1,3.20,30.80,Академический,https://www.an-nks.ru/search?view=filter-searc...
3,1,3.35,35.45,Академический,https://www.an-nks.ru/search?view=filter-searc...
4,1,3.46,34.95,Академический,https://www.an-nks.ru/search?view=filter-searc...
...,...,...,...,...,...
699,3,10.00,108.76,Ленинский,https://www.an-nks.ru/search?view=filter-searc...
700,3,10.30,108.10,Ленинский,https://www.an-nks.ru/search?view=filter-searc...
701,3,10.50,108.28,Ленинский,https://www.an-nks.ru/search?view=filter-searc...
702,4,10.60,114.30,Ленинский,https://www.an-nks.ru/search?view=filter-searc...


In [10]:
df['категория'] = df['цена_квартиры'].apply(category)

In [11]:
df.dtypes

количество_комнат     object
цена_квартиры        float64
площадь              float64
район                 object
ссылка                object
категория             object
dtype: object

In [12]:
df.loc[df['количество_комнат'] == 'Ст', 'количество_комнат'] = 0

In [13]:
df['количество_комнат'] = pd.to_numeric(df['количество_комнат'])
df.dtypes

количество_комнат      int64
цена_квартиры        float64
площадь              float64
район                 object
ссылка                object
категория             object
dtype: object

In [14]:
df.duplicated().sum()

137

In [15]:
df = df.drop_duplicates().reset_index(drop=True)
df.head(10)

,количество_комнат,цена_квартиры,площадь,район,ссылка,категория
0,1,2.85,27.10,Академический,https://www.an-nks.ru/search?view=filter-searc...,дешевая
1,1,2.95,27.10,Академический,https://www.an-nks.ru/search?view=filter-searc...,дешевая
2,1,3.20,30.80,Академический,https://www.an-nks.ru/search?view=filter-searc...,дешевая
3,1,3.35,35.45,Академический,https://www.an-nks.ru/search?view=filter-searc...,дешевая
4,1,3.46,34.95,Академический,https://www.an-nks.ru/search?view=filter-searc...,дешевая
5,1,3.68,37.25,Академический,https://www.an-nks.ru/search?view=filter-searc...,дешевая
6,1,3.79,40.85,Академический,https://www.an-nks.ru/search?view=filter-searc...,дешевая
7,1,3.82,41.15,Академический,https://www.an-nks.ru/search?view=filter-searc...,дешевая
8,1,3.91,42.00,Академический,https://www.an-nks.ru/search?view=filter-searc...,дешевая
9,1,3.94,42.50,Академический,https://www.an-nks.ru/search?view=filter-searc...,дешевая


In [16]:
features = df.drop(['цена_квартиры', 'ссылка'], axis=1)
target = df['ссылка']

In [17]:
features = pd.get_dummies(features)

In [18]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, random_state=12345, test_size=0.2)

In [19]:
features_train = features_train.reset_index(drop=True)
features_test = features_test.reset_index(drop=True)
target_train = target_train.reset_index(drop=True)
target_test = target_test.reset_index(drop=True)

In [20]:
features_test

,количество_комнат,площадь,район_Академический,район_Верх-Исетский,район_Железнодорожный,район_Ленинский,район_Октябрьский,район_Чкаловский,категория_дешевая,категория_премиальная,категория_средняя
0,2,57.00,0,0,1,0,0,0,1,0,0
1,1,36.00,0,0,0,0,0,1,1,0,0
2,2,48.30,0,0,0,0,0,1,1,0,0
3,2,60.00,0,0,0,0,1,0,0,0,1
4,3,73.40,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
109,2,48.00,0,0,0,0,0,1,1,0,0
110,1,42.51,0,0,0,1,0,0,1,0,0
111,0,24.00,0,0,0,0,0,1,1,0,0
112,1,40.65,0,0,0,1,0,0,0,0,1


In [21]:
features_test

,количество_комнат,площадь,район_Академический,район_Верх-Исетский,район_Железнодорожный,район_Ленинский,район_Октябрьский,район_Чкаловский,категория_дешевая,категория_премиальная,категория_средняя
0,2,57.00,0,0,1,0,0,0,1,0,0
1,1,36.00,0,0,0,0,0,1,1,0,0
2,2,48.30,0,0,0,0,0,1,1,0,0
3,2,60.00,0,0,0,0,1,0,0,0,1
4,3,73.40,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
109,2,48.00,0,0,0,0,0,1,1,0,0
110,1,42.51,0,0,0,1,0,0,1,0,0
111,0,24.00,0,0,0,0,0,1,1,0,0
112,1,40.65,0,0,0,1,0,0,0,0,1


In [22]:
df['ссылка'].value_counts()

https://fortis-development.ru/flats?result_mode=0&complex_visual_mode=0&property_visual_mode=1&is_free=1&complex=4                                                                                                                231
https://www.pik.ru/search/ekb/kosmonavtov11                                                                                                                                                                                        93
https://www.pik.ru/search/ekb/ip                                                                                                                                                                                                   90
https://жктихомиров.рф/flat/search-parameters                                                                                                                                                                                      49
https://www.an-nks.ru/search?view=filter-search&type=0&prev_type=0&district=Юго-

In [23]:
table = features.loc[df['ссылка'] == 'https://жктихомиров.рф/flat/search-parameters']
table

,количество_комнат,площадь,район_Академический,район_Верх-Исетский,район_Железнодорожный,район_Ленинский,район_Октябрьский,район_Чкаловский,категория_дешевая,категория_премиальная,категория_средняя
257,1,42.86,0,1,0,0,0,0,1,0,0
258,1,43.19,0,1,0,0,0,0,1,0,0
259,1,40.75,0,1,0,0,0,0,1,0,0
260,1,42.83,0,1,0,0,0,0,1,0,0
261,1,43.94,0,1,0,0,0,0,1,0,0
262,1,42.83,0,1,0,0,0,0,1,0,0
263,1,42.86,0,1,0,0,0,0,1,0,0
264,1,41.64,0,1,0,0,0,0,1,0,0
265,1,40.75,0,1,0,0,0,0,1,0,0
266,1,43.19,0,1,0,0,0,0,1,0,0


In [24]:
sk_model = KNeighborsClassifier(n_neighbors=len(features_train))
sk_model.fit(features_train, target_train)

KNeighborsClassifier(n_neighbors=453)

In [25]:
pr = sk_model.kneighbors([features_test.loc[0]], return_distance=False)
pr

array([[266, 152,  81, 220, 271, 159, 245, 270,  10, 277, 196,  11, 434,
         40, 155,  91,  66, 406, 243, 377,  78, 153,  13,  38, 124, 450,
        380, 397, 114,  28, 234, 392, 316, 258,  65, 369, 347, 439,  62,
        103, 311, 348, 294,   2, 398, 261, 423,  19, 288, 230, 123, 250,
        429, 278, 332, 432, 102, 117, 202, 125, 308, 116, 180, 276, 358,
         71, 240, 252, 229, 430, 128, 193, 115, 371, 269,  33,  35, 232,
         89, 425,  16,  47, 166, 363, 100, 403, 399,   4,  30,  20,  14,
         39, 214, 331, 333, 151, 204, 310, 334, 387, 323, 382, 138, 346,
        121,  63, 181, 195,  31,  77, 302, 112, 344,  88, 211, 161, 368,
        298, 139, 443,  60,  85, 290, 106, 366, 390, 321, 388, 108, 444,
        136, 313, 218,  82, 408,  72,  94, 239, 431, 314, 176, 219, 356,
         70, 337, 199, 110, 215, 173, 183, 307, 182, 119,  83,  18, 237,
        197, 286, 330, 206, 178, 194, 216, 336, 343, 301, 287, 374, 223,
        376, 188, 133, 227, 322, 433, 400, 169, 409

In [26]:
pr[0]

array([266, 152,  81, 220, 271, 159, 245, 270,  10, 277, 196,  11, 434,
        40, 155,  91,  66, 406, 243, 377,  78, 153,  13,  38, 124, 450,
       380, 397, 114,  28, 234, 392, 316, 258,  65, 369, 347, 439,  62,
       103, 311, 348, 294,   2, 398, 261, 423,  19, 288, 230, 123, 250,
       429, 278, 332, 432, 102, 117, 202, 125, 308, 116, 180, 276, 358,
        71, 240, 252, 229, 430, 128, 193, 115, 371, 269,  33,  35, 232,
        89, 425,  16,  47, 166, 363, 100, 403, 399,   4,  30,  20,  14,
        39, 214, 331, 333, 151, 204, 310, 334, 387, 323, 382, 138, 346,
       121,  63, 181, 195,  31,  77, 302, 112, 344,  88, 211, 161, 368,
       298, 139, 443,  60,  85, 290, 106, 366, 390, 321, 388, 108, 444,
       136, 313, 218,  82, 408,  72,  94, 239, 431, 314, 176, 219, 356,
        70, 337, 199, 110, 215, 173, 183, 307, 182, 119,  83,  18, 237,
       197, 286, 330, 206, 178, 194, 216, 336, 343, 301, 287, 374, 223,
       376, 188, 133, 227, 322, 433, 400, 169, 409, 296, 414, 43

In [27]:
target_train[pr[0]].drop_duplicates().reset_index(drop=True).loc[0:2]

0          https://www.pik.ru/search/ekb/kosmonavtov11
1    https://www.an-nks.ru/search?view=filter-searc...
2                     https://www.pik.ru/search/ekb/ip
Name: ссылка, dtype: object

In [28]:
def make_neighbors_data(rooms, area, place, category):
    sh = [
        [3, 19, 'академический', 'дешевая'],
        [4, 20, 'верх-исетский', 'премиальная'],
        [6, 22, 'железнодорожный', 'средняя'],
        [7, 23, 'ленинский', 'дешевая'],
        [8, 24, 'октябрьский', 'премиальная'],
        [9, 25, 'чкаловский', 'средняя'],
        [rooms, area, place, category]
    ]
    col = ['количество_комнат', 'площадь', 'район', 'категория']
    data = pd.DataFrame(data=sh, columns=col)
    data = pd.get_dummies(data)
    return data.loc[6, :]

In [29]:
def near_neighbors(model, target_train,  rooms, area, place, category):
    for_model_data = make_neighbors_data(rooms, area, place, category)
    near = model.kneighbors([for_model_data], return_distance=False)[0]
    res = target_train[near].drop_duplicates().reset_index(drop=True)
    return res.loc[0:2]

In [30]:
near_neighbors(sk_model, target_train, 4, 27, 'чкаловский', 'дешевая')

0    https://www.an-nks.ru/search?view=filter-searc...
1    https://fortis-development.ru/flats?result_mod...
2    https://www.an-nks.ru/search?view=filter-searc...
Name: ссылка, dtype: object

In [31]:
a = [make_neighbors_data(4, 27, 'чкаловский', 'дешевая')]
a

[количество_комнат         4
 площадь                  27
 район_академический       0
 район_верх-исетский       0
 район_железнодорожный     0
 район_ленинский           0
 район_октябрьский         0
 район_чкаловский          1
 категория_дешевая         1
 категория_премиальная     0
 категория_средняя         0
 Name: 6, dtype: int64]

In [32]:
b = sk_model.kneighbors(a, return_distance=False)[0]
b

array([386, 264, 263, 145, 418, 189,  43, 212, 422, 372,  34, 142, 120,
       419,  17, 272,  52, 192, 187, 355, 340, 168, 351,  64, 280, 149,
       295,  86, 297, 126, 262, 335,  24, 186, 148,  12,  96, 244,   1,
       274, 326, 285, 235, 165, 303,  67, 315, 101, 130, 207,  15, 300,
        45, 437,  76, 254,   3,  87, 131, 306, 447, 325, 448,  32,  59,
        97, 365,  50, 381, 391, 357, 268, 241, 329, 190, 226, 379,  37,
        22, 426,  27, 364, 279,  46, 224, 113,  25, 167, 440, 191, 251,
        84, 283, 267,  80, 395, 260, 256, 253, 349, 157, 360, 259,  69,
       281, 413, 291,  73,  55, 367, 209,  26,  54, 339, 164, 451, 309,
       353,  92, 312, 265, 320, 156, 228, 317, 420, 111, 421, 417,  98,
       217, 385, 378, 275, 293, 305,  61, 162, 449, 238, 242, 179, 129,
        79, 109, 370,  51,  49, 205, 441, 208, 231, 134, 184, 427, 327,
       140, 410, 328, 445, 446, 170, 201, 203, 174, 416,  95,  99, 299,
       213, 282,  21, 352, 127, 424, 225, 375, 383, 402, 345,   

In [33]:
best_neighbors = KNeighborsClassifier(n_neighbors=len(features))
best_neighbors.fit(features, target)

KNeighborsClassifier(n_neighbors=567)

In [34]:
#with open('neighbors.pkl' , 'wb') as f:
    #pickle.dump(best_neighbors, f)

In [35]:
target.to_csv('url.csv', index=False)

In [36]:
a = pd.read_csv('url.csv')['ссылка']
a

In [38]:
target

0      https://www.an-nks.ru/search?view=filter-searc...
1      https://www.an-nks.ru/search?view=filter-searc...
2      https://www.an-nks.ru/search?view=filter-searc...
3      https://www.an-nks.ru/search?view=filter-searc...
4      https://www.an-nks.ru/search?view=filter-searc...
                             ...                        
562    https://www.an-nks.ru/search?view=filter-searc...
563    https://www.an-nks.ru/search?view=filter-searc...
564    https://www.an-nks.ru/search?view=filter-searc...
565    https://www.an-nks.ru/search?view=filter-searc...
566    https://www.an-nks.ru/search?view=filter-searc...
Name: ссылка, Length: 567, dtype: object